In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/jw3514/Work/ASD_Circuits/src')
from ASD_Circuits import *
from tabulate import tabulate

In [ ]:
HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol, allen_mouse_genes = LoadGeneINFO()
Z2Mat = pd.read_csv("../dat/allen-mouse-exp//AllenMouseBrain_Z2bias.csv", index_col=0)
allen_mouse_genes = Z2Mat.index.values
BGMR = pd.read_csv("~/Work/Resources/MutationRate_20170710_rate.txt", delimiter="\t")
BGMR["Entrez"] = [int(GeneSymbol2Entrez.get(x, -1)) for x in BGMR["GeneName"].values]
BGMR = BGMR[BGMR["Entrez"].isin(allen_mouse_genes)]
BGMR.index = BGMR["Entrez"].values
BGMR["Mutability_Dmg"] = BGMR["p_LGD"] + BGMR["prevel_0.5"]
BGMR["Mutability_syn"] = BGMR["p_synonymous"]

In [ ]:
BGMR.head(2)

### Correlation between Mut Rate and Mut cont of Sibling

In [ ]:
SiblingDF = pd.read_csv("../scripts/dat/ASD_Siblings.csv")
SiblingDF.head(2)

In [ ]:
Mut_rate = []
Mut_count = []
for i, row in SiblingDF.iterrows():
    entrez = row["Entrez"]
    Muts = row["N_LGD"] + row["N_Dmis"]
    #print(entrez)
    if Muts > 0 and entrez in BGMR.index.values:
        Mut_rate.append(BGMR.loc[entrez, "Mutability_Dmg"])
        Mut_count.append(Muts)

In [ ]:
print(len(Mut_count))
plt.scatter(Mut_count, Mut_rate)
pearsonr(Mut_count, Mut_rate)

### Simulations

In [ ]:
meta = pd.read_csv("/home/jw3514/Work/ASD_Circuits/scripts/dat/meta_denovo_simulations.csv", 
                   index_col="index")

In [ ]:
meta.head(2)

In [ ]:
def xxx(genes, values, BGMR):
    
    Mut_rate = []
    Mut_count = []
    for entrez, value in zip(genes, values):
        #print(entrez, value)
        Muts = np.sum([int(x) for x in value.split(",")])
        #print(Muts)
        if Muts > 0 and int(entrez) in BGMR.index.values:
            Mut_rate.append(BGMR.loc[int(entrez), "Mutability_Dmg"])
            Mut_count.append(Muts)
    print(len(Mut_count))
    plt.scatter(Mut_count, Mut_rate)
    print(pearsonr(Mut_count, Mut_rate))
    plt.show()
    
for i, row in meta.iterrows():
    xxx(meta.columns, row, BGMR)
    if i > 5:
        break